In [2]:
##This is adapted from https://github.com/tkipf/gae/blob/master/gae/train.py##

import time
import os

# Train on CPU (hide GPU) due to memory constraints
# os.environ['CUDA_VISIBLE_DEVICES'] = ""


import numpy as np
import scipy.sparse as sp

import torch

from gae.gae.optimizer import OptimizerAE, OptimizerVAE
from gae.gae.input_data import load_data
from gae.gae.model import GCNModelAE, GCNModelVAE
from gae.gae.preprocessing import preprocess_graph, construct_feed_dict, sparse_to_tuple, mask_test_edges

torch.manual_seed(0)

In [2]:
# Settings

flags = tf.compat.v1.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 32, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 16, 'Number of units in hidden layer 2.')
flags.DEFINE_float('weight_decay', 0., 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')

flags.DEFINE_string('model', 'gcn_ae', 'Model string.')
flags.DEFINE_string('dataset', 'cora', 'Dataset string.')
flags.DEFINE_integer('features', 1, 'Whether to use features (1) or not (0).')

model_str = FLAGS.model
dataset_str = FLAGS.dataset


UnrecognizedFlagError: Unknown command line flag 'f'

In [11]:
FLAGS.learning_rate

UnrecognizedFlagError: Unknown command line flag 'f'